In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


#enable IPython to display matplolib graphs
%matplotlib inline

from sklearn import tree


In [2]:
from ML_classes.DTModel import DTModel
from ML_classes.LinearModel import LinearModel
from ML_classes.LSTMModel import LSTMModel
from ML_classes.MLPModel import MLPModel



2022-03-28 13:23:30.549569: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 13:23:30.549589: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
weather_data = pd.read_csv("./data/weather_hourly_darksky.csv")
weather_data["time"] = pd.to_datetime(weather_data["time"])
weather_data["temperature"] = pd.to_numeric(weather_data["temperature"], downcast="float", errors="coerce")


In [4]:
#if we are to interpolate temperature for halfhour

In [5]:
df = pd.read_csv('./data/ouput.csv')
df['tstp'] = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in df['tstp']]
#df["tstp"] = pd.to_datetime(df["tstp"])
df["energy(kWh/hh)"] = pd.to_numeric(df["energy(kWh/hh)"], downcast="float", errors="coerce")


In [6]:
houses_with_temperature_correlation = ["MAC000153","MAC000168","MAC000169","MAC000173"]

In [7]:
#for singular house testing
lclid_list = df['LCLid'].unique()
def find_household(identification):
    affluenthh_filt = df["LCLid"] == identification
    affluent_hh_data = df[affluenthh_filt]
    return affluent_hh_data

hh = find_household("MAC000153")
hh.pop("LCLid")

#making data hourly or not
hh = hh.set_index("tstp")
hh = hh.resample("H").sum()
hh = hh.reset_index()

#use of weather data
weather_set = weather_data.set_index("time")
weather_set = weather_set[str(hh["tstp"].min()):str(hh["tstp"].max())]
weather_set = weather_set.reset_index()
hh["temperature"] = weather_set["temperature"]
# weather data doesnt necessarily contain data for all dates
hh.dropna(axis=0,how='any',subset=["temperature"], inplace=True)

hh_max = hh['energy(kWh/hh)'].max()
hh['energy(kWh/hh)'] = hh['energy(kWh/hh)'].apply(lambda x: x / hh_max)
hh['temperature'] = hh['temperature'].apply(lambda x: x / hh_max)

In [8]:
## DT
#train for all houses
#lclid_list = df['LCLid'].unique().tolist()
#print(type(lclid_list))
mse = []
rmse = []
mae = []
mape = []
for lclid in houses_with_temperature_correlation :
    #print(lclid)


    hh = find_household(lclid)
    hh.pop("LCLid")
    #hourly
    hh = hh.set_index("tstp")
    hh = hh.resample("H").sum()
    hh = hh.reset_index()
    
    #use of weather data
    weather_set = weather_data.set_index("time")
    weather_set = weather_set[str(hh["tstp"].min()):str(hh["tstp"].max())]
    weather_set = weather_set.reset_index()
    hh["temperature"] = weather_set["temperature"]
    # weather data doesnt necessarily contain data for all dates
    hh.dropna(axis=0,how='any',subset=["temperature"], inplace=True)

    #normalization
    hh_max = hh['energy(kWh/hh)'].max()
    hh['energy(kWh/hh)'] = hh['energy(kWh/hh)'].apply(lambda x: x / hh_max)
    hh['temperature'] = hh['temperature'].apply(lambda x: x / hh_max)
    
    
    # Initiating the class
    DT = DTModel(
    data = hh,
    Y_var = 'energy(kWh/hh)',
    lag = 24,
    epochs = 20,
    batch_size = 256,
    train_test_split = 0.15,
    data_creator= "temperature"
    )
    #training
    DT.DTModel()
    
    mse.append(DT.evaluateMSE())
    rmse.append(DT.evaluateRMSE())
    mae.append(DT.evaluateMAE())
    mape.append(DT.evaluateMAPE())
houses = pd.Series(data=houses_with_temperature_correlation,name="lclid")
mse = pd.Series(data=mse,name="mse")
rmse = pd.Series(data=rmse,name="rmse")
mae = pd.Series(data=mae,name="mae")
mape = pd.Series(data=mape,name="mape")

res_df = pd.merge(houses, mse, right_index=True, left_index=True)
res_df["rmse"] = rmse
res_df["mae"] = mae
res_df["mape"] = mape

res_df

,lclid,mse,rmse,mae,mape
0,MAC000153,0.011693,0.108135,0.073251,54.616649
1,MAC000168,0.010656,0.103230,0.040350,119.923288
2,MAC000169,0.009806,0.099024,0.057507,85.940443
3,MAC000173,0.016000,0.126492,0.077987,51.774568


In [9]:
## perceptron
#train for all houses
#lclid_list = df['LCLid'].unique().tolist()
#print(type(lclid_list))
mse = []
rmse = []
mae = []
mape = []
for lclid in houses_with_temperature_correlation :
    #print(lclid)


    hh = find_household(lclid)
    hh.pop("LCLid")
    #hourly
    hh = hh.set_index("tstp")
    hh = hh.resample("H").sum()
    hh = hh.reset_index()
    
    #use of weather data
    weather_set = weather_data.set_index("time")
    weather_set = weather_set[str(hh["tstp"].min()):str(hh["tstp"].max())]
    weather_set = weather_set.reset_index()
    hh["temperature"] = weather_set["temperature"]
    # weather data doesnt necessarily contain data for all dates
    hh.dropna(axis=0,how='any',subset=["temperature"], inplace=True)

    #normalization
    hh_max = hh['energy(kWh/hh)'].max()
    hh['energy(kWh/hh)'] = hh['energy(kWh/hh)'].apply(lambda x: x / hh_max)
    hh['temperature'] = hh['temperature'].apply(lambda x: x / hh_max)
    
    
    # Initiating the class
    Linear = LinearModel(
    data = hh,
    Y_var = 'energy(kWh/hh)',
    lag = 24,
    epochs = 20,
    batch_size = 256,
    train_test_split = 0.15,
    data_creator= "temperature"
    )
    #training
    Linear.LinearModel()
    
    mse.append(Linear.evaluateMSE())
    rmse.append(Linear.evaluateRMSE())
    mae.append(Linear.evaluateMAE())
    mape.append(Linear.evaluateMAPE())
houses = pd.Series(data=houses_with_temperature_correlation,name="lclid")
mse = pd.Series(data=mse,name="mse")
rmse = pd.Series(data=rmse,name="rmse")
mae = pd.Series(data=mae,name="mae")
mape = pd.Series(data=mape,name="mape")

res_df = pd.merge(houses, mse, right_index=True, left_index=True)
res_df["rmse"] = rmse
res_df["mae"] = mae
res_df["mape"] = mape



Epoch 1/20


2022-03-28 13:23:44.819893: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-28 13:23:44.820180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 13:23:44.820231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-28 13:23:44.820278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-28 13:23:44.820324: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

66/66 [==============================] - 0s 2ms/step - loss: 0.1089 - val_loss: 0.1103
Epoch 2/20
66/66 [==============================] - 0s 985us/step - loss: 0.0752 - val_loss: 0.0742
Epoch 3/20
66/66 [==============================] - 0s 950us/step - loss: 0.0560 - val_loss: 0.0517
Epoch 4/20
66/66 [==============================] - 0s 936us/step - loss: 0.0458 - val_loss: 0.0415
Epoch 5/20
66/66 [==============================] - 0s 918us/step - loss: 0.0402 - val_loss: 0.0362
Epoch 6/20
66/66 [==============================] - 0s 924us/step - loss: 0.0365 - val_loss: 0.0327
Epoch 7/20
66/66 [==============================] - 0s 936us/step - loss: 0.0335 - val_loss: 0.0300
Epoch 8/20
66/66 [==============================] - 0s 962us/step - loss: 0.0310 - val_loss: 0.0278
Epoch 9/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0288 - val_loss: 0.0259
Epoch 10/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0269 - val_loss: 0.0242
Epoch 11/20
66/6

In [10]:
res_df

,lclid,mse,rmse,mae,mape
0,MAC000153,0.014503,0.120428,0.090246,77.275519
1,MAC000168,0.070989,0.266437,0.192602,1346.581671
2,MAC000169,0.196701,0.443510,0.316041,561.607636
3,MAC000173,0.082455,0.287149,0.195174,159.779783


In [11]:
## SLP
#train for all houses
#lclid_list = df['LCLid'].unique().tolist()
#print(type(lclid_list))
mse = []
rmse = []
mae = []
mape = []
for lclid in houses_with_temperature_correlation :
    #print(lclid)


    hh = find_household(lclid)
    hh.pop("LCLid")
    #hourly
    hh = hh.set_index("tstp")
    hh = hh.resample("H").sum()
    hh = hh.reset_index()
    
    #use of weather data
    weather_set = weather_data.set_index("time")
    weather_set = weather_set[str(hh["tstp"].min()):str(hh["tstp"].max())]
    weather_set = weather_set.reset_index()
    hh["temperature"] = weather_set["temperature"]
    # weather data doesnt necessarily contain data for all dates
    hh.dropna(axis=0,how='any',subset=["temperature"], inplace=True)

    #normalization
    hh_max = hh['energy(kWh/hh)'].max()
    hh['energy(kWh/hh)'] = hh['energy(kWh/hh)'].apply(lambda x: x / hh_max)
    hh['temperature'] = hh['temperature'].apply(lambda x: x / hh_max)
    
    
    # Initiating the class
    MLP = MLPModel(
    data = hh,
    Y_var = 'energy(kWh/hh)',
    lag = 24,
    layer_depths = [30],
    layer_count = 1,
    epochs = 20,
    batch_size = 256,
    train_test_split = 0.15,
    data_creator= "temperature"
    )
    #training
    MLP.MLPModel()
    
    mse.append(MLP.evaluateMSE())
    rmse.append(MLP.evaluateRMSE())
    mae.append(MLP.evaluateMAE())
    mape.append(MLP.evaluateMAPE())
houses = pd.Series(data=houses_with_temperature_correlation,name="lclid")
mse = pd.Series(data=mse,name="mse")
rmse = pd.Series(data=rmse,name="rmse")
mae = pd.Series(data=mae,name="mae")
mape = pd.Series(data=mape,name="mape")

res_df = pd.merge(houses, mse, right_index=True, left_index=True)
res_df["rmse"] = rmse
res_df["mae"] = mae
res_df["mape"] = mape



Epoch 1/20
66/66 [==============================] - 0s 2ms/step - loss: 0.1126 - val_loss: 0.0293
Epoch 2/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0231 - val_loss: 0.0217
Epoch 3/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0174 - val_loss: 0.0176
Epoch 4/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0148 - val_loss: 0.0156
Epoch 5/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0134 - val_loss: 0.0146
Epoch 6/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0124 - val_loss: 0.0138
Epoch 7/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0116 - val_loss: 0.0130
Epoch 8/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0110 - val_loss: 0.0124
Epoch 9/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0105 - val_loss: 0.0118
Epoch 10/20
66/66 [==============================] - 0s 1ms/step - loss: 0.0101 - val_loss: 0.0112
Epoch 11/20
66/66 [

In [12]:
res_df

,lclid,mse,rmse,mae,mape
0,MAC000153,0.009036,0.095057,0.068704,50.478544
1,MAC000168,0.016740,0.129382,0.091074,589.174237
2,MAC000169,0.005340,0.073077,0.047794,68.755469
3,MAC000173,0.035163,0.187519,0.133220,108.659656


In [13]:
MLP.model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 30)                1470      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 1,501
Trainable params: 1,501
Non-trainable params: 0
_________________________________________________________________


In [14]:
## SLP
#train for all houses
#lclid_list = df['LCLid'].unique().tolist()
#print(type(lclid_list))
mse = []
rmse = []
mae = []
mape = []
for lclid in houses_with_temperature_correlation :
    #print(lclid)


    hh = find_household(lclid)
    hh.pop("LCLid")
    #hourly
    hh = hh.set_index("tstp")
    hh = hh.resample("H").sum()
    hh = hh.reset_index()
    
    #use of weather data
    weather_set = weather_data.set_index("time")
    weather_set = weather_set[str(hh["tstp"].min()):str(hh["tstp"].max())]
    weather_set = weather_set.reset_index()
    hh["temperature"] = weather_set["temperature"]
    # weather data doesnt necessarily contain data for all dates
    hh.dropna(axis=0,how='any',subset=["temperature"], inplace=True)

    #normalization
    hh_max = hh['energy(kWh/hh)'].max()
    hh['energy(kWh/hh)'] = hh['energy(kWh/hh)'].apply(lambda x: x / hh_max)
    hh['temperature'] = hh['temperature'].apply(lambda x: x / hh_max)
    
    
    # Initiating the class
    LSTM = LSTMModel(
    data = hh,
    Y_var = 'energy(kWh/hh)',
    lag = 24,
    LSTM_layer_depth = 50,
    epochs = 20,
    batch_size = 256,
    train_test_split = 0.15,
    data_creator= "temperature"
    )
    #training
    LSTM.LSTModel()
    
    mse.append(LSTM.evaluateMSE())
    rmse.append(LSTM.evaluateRMSE())
    mae.append(LSTM.evaluateMAE())
    mape.append(LSTM.evaluateMAPE())
houses = pd.Series(data=houses_with_temperature_correlation,name="lclid")
mse = pd.Series(data=mse,name="mse")
rmse = pd.Series(data=rmse,name="rmse")
mae = pd.Series(data=mae,name="mae")
mape = pd.Series(data=mape,name="mape")

res_df = pd.merge(houses, mse, right_index=True, left_index=True)
res_df["rmse"] = rmse
res_df["mae"] = mae
res_df["mape"] = mape


Epoch 1/20
66/66 [==============================] - 3s 30ms/step - loss: 0.0286 - val_loss: 0.0377
Epoch 2/20
66/66 [==============================] - 2s 28ms/step - loss: 0.0284 - val_loss: 0.0267
Epoch 3/20
66/66 [==============================] - 2s 28ms/step - loss: 0.0257 - val_loss: 0.0257
Epoch 4/20
66/66 [==============================] - 2s 28ms/step - loss: 0.0251 - val_loss: 0.0228
Epoch 5/20
66/66 [==============================] - 2s 28ms/step - loss: 0.0245 - val_loss: 0.0221
Epoch 6/20
66/66 [==============================] - 2s 28ms/step - loss: 0.0242 - val_loss: 0.0215
Epoch 7/20
66/66 [==============================] - 2s 28ms/step - loss: 0.0239 - val_loss: 0.0213
Epoch 8/20
66/66 [==============================] - 2s 29ms/step - loss: 0.0238 - val_loss: 0.0209
Epoch 9/20
66/66 [==============================] - 2s 28ms/step - loss: 0.0235 - val_loss: 0.0211
Epoch 10/20
66/66 [==============================] - 2s 29ms/step - loss: 0.0234 - val_loss: 0.0208
Epoch 11/

In [15]:
res_df

,lclid,mse,rmse,mae,mape
0,MAC000153,0.019041,0.137989,0.093147,71.063019
1,MAC000168,0.006507,0.080664,0.045082,216.246500
2,MAC000169,0.007498,0.086591,0.058565,78.223755
3,MAC000173,0.018462,0.135874,0.095774,64.023209
